## Notebook para ilustrar diferenças entre os corpus 'treebank' e 'floresta' e por que não é possível realizar parsing na gramática probabilística induzida de 'floresta'

Ao final deste novebook é proposta uma Atividade 10 complementar.

Instalando a versão mais recente do NLTK (3.5), o default do colab é 3.2. Esta foi uma tentativa de eliminar o erro (instalando a versão mais recente do NLTK), mas não resolveu, pois o problema  reside, na verdade, no corpus 'floresta'.

In [ ]:
!pip install --upgrade nltk

     |████████████████████████████████| 1.4MB 2.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434672 sha256=3035f25f96bd62285d133ce5401a3f5fc7b0072fa9f427d2d3fc8b3d48d360a4
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


Fazendo download dos corpus 'floresta' e 'treebank'

In [ ]:
import nltk
nltk.download('floresta')
nltk.download('treebank')

[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Unzipping corpora/floresta.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

Imprimindo sentenças

In [ ]:
from nltk.corpus import floresta
from nltk.corpus import treebank

In [ ]:
print("Corpus Floresta")
x=floresta.sents()
#imprime as 3 primeiras sentenças 
for i in range(3):
  print(x[i])
#imprime as sentenças de índices 2000 a 2002
for i in range(2000,2003):
  print(x[i])

print("\n")
print("Corpus Treebank")
x=treebank.sents()
#imprime as 3 primeiras sentenças 
for i in range(3):
  print(x[i])
#imprime as sentenças de índices 2000 a 2002
for i in range(2000,2003):
  print(x[i])

Corpus Floresta
['Um', 'revivalismo', 'refrescante']
['O', '7_e_Meio', 'é', 'um', 'ex-libris', 'de', 'a', 'noite', 'algarvia', '.']
['É', 'uma', 'de', 'as', 'mais', 'antigas', 'discotecas', 'de', 'o', 'Algarve', ',', 'situada', 'em', 'Albufeira', ',', 'que', 'continua', 'a', 'manter', 'os', 'traços', 'decorativos', 'e', 'as', 'clientelas', 'de', 'sempre', '.']
['Mas', 'as', 'coisas', 'em', 'Espanha', 'não', 'estão', 'nada', 'fáceis', '.']
['A', 'derrapagem', 'de', 'a', 'economia', 'acentua-', 'se', '.']
['Os', 'números', 'de', 'o', 'desemprego', 'conhecidos', 'em', 'a', 'semana', 'passada', 'são', 'desastrosos', ';']


Corpus Treebank
['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.']
['Rudolph', 'Agnew', ',', '55', 'years', 'old', 'and', 'former', 'chairman', 'of', 'Consolidated', 'Gol

Cada treebank possui um ou mais arquivos de texto (fileid) de onde as estruturas da linguagem (como sentenças, palavras, árvores de parsing etc) podem ser acessadas.

Além disto, o corpus floresta tem algumas árvores de parsing com problemas. Para evitar mensagens de erro associadas a tais árvores com problema, você pode desabilitá-las  temporariamente usando as funções auxiliares a seguir.

In [ ]:
#funções auxiliares para controle de saída de erro padrão
import os
import sys
def disable_stderr():
  r=sys.stderr
  f = open(os.devnull, 'w')
  sys.stderr = f
  return r

def enable_stderr(f):
  sys.stderr=f

In [ ]:
#desabilitando std_err. Para habilitar, basta chamar enable_stderr(r)
r=disable_stderr()

O corpus treebank tem muitos fileids (199), enquanto que o corpus floresta possui apenas 1.

In [ ]:
#computa e imprime totais de arquivos de texto de cada corpus
lf=len(floresta.fileids())
lt=len(treebank.fileids())

print('len(floresta) ', lf)
print('len(treebank) ', lt)

len(floresta)  1
len(treebank)  199


Vamos agora calcular os totais de árvores de parsing de cada corpus de texto.

Ao executar o código abaixo, é possível perceber que os 3 primeiros textos do corpus 'treebank' contem apenas 33 árvores de parsing (que foram usadas para treinar a gramática do notebook original), enquanto que o único texto do corpus 'floresta' possui 9266 árvores! 

In [ ]:
#contando total de árvores nos 3 primeiros textos do corpus 'treebank' (apenas estes 3 textos foram utilizados no notebook NLTK-new.ipynb)
acc=0
for i in treebank.fileids()[:3]:
  acc = acc+len(treebank.parsed_sents(i))
print('total_arvores treebank=',acc)


#contando total de árvores no único texto disponível do corpus 'floresta'
acc=0
for i in floresta.fileids()[:1]:
  lf = len(floresta.parsed_sents(i))
  acc = acc+lf
print('total_arvores floresta=',acc)

total_arvores treebank= 33
total_arvores floresta= 9266


In [ ]:
#habilitando novamente std_err para que as mensagens de erro voltem a ser impressas
enable_stderr(r)

Vamos agora inspecionar os totais de palavras de cada corpus. Percebam que o corpus floresta é bem mais rico em número de palavras.

In [ ]:
print("floresta.words=",len(floresta.words()), "\ntreebank.words=", len(treebank.words()))

floresta.words= 211852 
treebank.words= 100676


Vamos agora aprender a percorrer as árvores de parsing do corpus 'floresta' e normalizar as regras de produção, evitando aquelas que não podem ser normalizadas.  

Para fazer isso, vamos empregar tratamento de exceções. Ao executar o código, é possível perceber que apenas uma pequena quantidade de árvores de  parsing não pode ser normalizada.

In [ ]:
#desabilitando std_err. Para habilitar, basta chamar enable_stderr(r)
r=disable_stderr()

In [ ]:
from nltk import treetransforms
from nltk import induce_pcfg
from nltk import Nonterminal

#contadores para árvores ok e para árvores com falha
ok=0;
falha=0;

psents=floresta.parsed_sents()
productions=[]

Processando as árvores de parsing com tratamento de exceção. É possível perceber que algumas das árvores não podem ser normalizadas para a forma normal de chomsky.

In [ ]:
for tree in psents:
  tree.collapse_unary(collapsePOS = False)# Transform branches A-B-C into A-(B+C)
  try:
    tree.chomsky_normal_form(horzMarkov = 2)# Transform A->(B,C,D) into A->B,(C+D)->D
    productions += tree.productions()     
    ok=ok+1
  except:
    print(tree,"\n")
    falha=falha+1

print("falha=", falha)
print("ok=", ok)

(S
  Em/H+prp
  o/>N+art
  imenso/>N+adj
  desacerto/H+n
  que/SUBJ+pron-indp
  foi/P+v-fin
  a/>N+art
  defesa/H+n
  de/H+prp
  o/>N+art
  Penafiel/H+prop
  ,/,
  o/>N+art
  capitão/H+n
  Vasco/N<+prop
  foi/P+v-fin
  o/>N+art
  homem/H+n
  que/SUBJ+pron-indp
  ainda/ADVL+adv
  segurou/P+v-fin
  as/>N+art
  pontas/H+n
  ./.) 

(S
  José_Eduardo_dos_Santos/CJT+prop
  e/CO+conj-c
  Jonas_Savimbi/CJT+prop
  reúnem-/P+v-fin
  se/ACC+pron-pers
  em/H+prp
  Libreville/P<+prop
  ,/,
  concordando/P+v-ger
  que/SUB+conj-s
  um/>N+art
  Governo_de_Unidade_e_Reconciliação/H+prop
  esteja/P+v-fin
  formado/SC+v-pcp
  até/H+prp
  Junho/CJT+n
  ou/CO+conj-c
  Julho/CJT+n
  ./.) 

(S
  De/H+prp
  um/>N+art
  modo/H+n
  geral/N<+adj
  ,/,
  os/>N+art
  animais/H+n
  domésticos/N<+adj
  que/SUBJ+pron-indp
  vivem/P+v-fin
  em/H+prp
  o/>N+art
  campo/H+n
  têm/P+v-fin
  uma/>N+art
  existência/H+n
  mais/>A+adv
  feliz/H+adj
  do_que/COM+conj-s
  os/>N+art
  citadinos/H+n
  ./.) 

(S Sorteio/SUBJ+n s

Inspecionando as 10 primeiras regras de produção normalizadas

In [ ]:
productions[:10]

[UTT+np -> >N+art UTT+np|<H+n-N<+adj>,
 >N+art -> 'Um',
 UTT+np|<H+n-N<+adj> -> H+n N<+adj,
 H+n -> 'revivalismo',
 N<+adj -> 'refrescante',
 STA+fcl -> SUBJ+np STA+fcl|<P+v-fin-SC+np>,
 SUBJ+np -> >N+art H+prop,
 >N+art -> 'O',
 H+prop -> '7_e_Meio',
 STA+fcl|<P+v-fin-SC+np> -> P+v-fin STA+fcl|<SC+np-.>]

Inspecionando apenas as regras de produção começando pelo símbolo não terminal S.

In [ ]:
for p in productions:
  if p.lhs() == Nonterminal('S'):
    print(p)

S -> ('de', 'H+prp') ('Philip_Setbon', 'P<+prop')
S -> ('De', 'H+prp') ('Dan_Aykroyd', 'P<+prop')


Induzindo a gramática probabilística a partir das regras de produção normalizadas e imprimindo as regras de produção probabilísticas resultantes.

In [ ]:
S = Nonterminal('S')
grammar = induce_pcfg(S, productions)

In [ ]:
for p in grammar.productions():
    print(p)

Streaming output truncated to the last 5000 lines.
ACC+n -> 'superpartido' [0.00184502]
H+n -> 'Lideranças' [2.91758e-05]
P+v-inf -> 'apostar' [0.00035137]
H+n -> 'legenda' [2.91758e-05]
N<+adj -> 'social-democrata' [0.000196155]
PRED+ap -> H+adj N<+pp [0.0454545]
H+adj -> 'Confiante' [0.000600601]
H+n -> 'superpartido' [2.91758e-05]
H+n -> 'tucanos' [2.91758e-05]
STA+fcl|<P+v-fin-OC+adj> -> P+v-fin STA+fcl|<OC+adj-ACC+icl> [0.125]
STA+fcl|<OC+adj-ACC+icl> -> OC+adj STA+fcl|<ACC+icl-.> [1.0]
OC+adj -> 'perigoso' [0.0151515]
P<+np -> >N+art P<+np|<H+cu-N<+adj> [9.00374e-05]
P<+np|<H+cu-N<+adj> -> H+cu N<+adj [1.0]
CJT+n -> 'senadores' [0.00245098]
N<+adj -> 'pefelistas' [0.000196155]
SUBJ+prop -> 'Grava' [0.000900901]
H+n -> 'rompimentos' [2.91758e-05]
H+n -> 'feixes' [2.91758e-05]
N<+adj -> 'musculares' [0.000196155]
H+n -> 'músculos' [2.91758e-05]
H+n -> 'coxa' [8.75274e-05]
P<+prop -> 'Viola' [0.000584454]
H+n -> 'músculo' [2.91758e-05]
H+adv -> 'de_trás' [0.00123001]
CJT+fcl -> SUBJ

Acessando uma sentença do corpus 'floresta' para fins de teste.

In [ ]:
from nltk.parse import pchart
new_parser = pchart.InsideChartParser(grammar)
sent = floresta.parsed_sents()[23].leaves()
print(sent)


['A', 'zona', 'lúdica', ',', 'com', 'os', 'divertimentos', ',', 'áreas', 'comerciais', 'de', '«', 'souvenirs', 'e', 'de', 'restauração', ',', 'compreende', 'espaços', 'distintos', 'para', 'os', 'vários', 'temas', ',', 'ainda', 'em', 'análise', ',', 'tais', 'como', 'Portugal', ',', 'Japão', ',', 'Brasil', ',', 'África', 'e', 'Far-West', '.']


Ao tentarmos realizar o parsing da sentença acima, verificamos que há uma mensagem de erro indicando problema nas regras de produção. Conforme veremos mais abaixo, há regras na gramática probabilística em que o lado direito está vazio e o lado esquerdo contém caracteres especiais. Provavelmente, o corpus 'floresta' tem árvores inconsistentes que precisariam ser filtradas, mas esta tarefa está fora do escopo do que se pretendia com a Atividade 10. Desta forma, encerraremos neste ponto.

In [ ]:
new_parser.parse_all(sent)

IndexError: ignored

Mostrando que existem regras de produção inválidas (lado direito sem símbolos), o que impede realizar o parsing da sentença fornecida.

In [ ]:
prod_str=[]
for prod in grammar.productions():
    if len(prod.rhs()) > 0 :
       prod_str.append(str(prod))
    else:
      print(prod)

»" ->  [1.0]
-- ->  [0.998454]
.. ->  [1.0]
... ->  [1.0]
». ->  [1.0]
"« ->  [1.0]


## Exemplos de uso de uma palavra
Vamos criar uma função recebe uma palavra e um tamanho de contexto (em número de caracteres à esquerda e à direita),  e produz como resposta exemplos de uso da tal palavra (pode também ser útil para saber a concordância).

In [ ]:
def exemplos_uso(word, context=30):
    print(f"palavra: {word}, contexto: {context} caracteres")
    for sent in floresta.sents():
        if word in sent:
            pos = sent.index(word)
            left = " ".join(sent[:pos])
            right = " ".join(sent[pos + 1:])
            print(f"{left[-context:]} '{word}' {right[:context]}")

exemplos_uso("dar")

palavra: dar, contexto: 30 caracteres
anduru , foi o suficiente para 'dar' a volta a o resultado .
1. O PÚBLICO veio 'dar' a a imprensa diária portuguesa
A fartura de pensamento pode 'dar' maus resultados e nós não quer
Começa a 'dar' resultados a política de a Uni
ial começar a incorporar- lo e 'dar' forma a um ' site ' que tem se
r com Constantino para ele lhe 'dar' também os papéis assinados .
va a brincar , pois não lhe ia 'dar' procuração nenhuma enquanto nã
érica como o antídoto capaz de 'dar' sentido a o seu enorme poder .
Os agentes decidiram 'dar' vantagem a os raptores por est
ducação_Tecnológica , sem lhes 'dar' qualquer tipo de formação .
o CDS , a oposição decidiu não 'dar' acordo para que a Lei_Orgânica
 em vésperas de o século XXI , 'dar' novos argumentos os de a esper
isfação que vemos a Plataforma 'dar' razão « a_posteriori » a Álvar
 intervenção e funcionamento , 'dar' razão a Cunhal é também dar ra
« Parecia que tinha acabado de 'dar' um aperto de mão a um velho am
P

#Agora é a sua vez
Com base na função acima, crie uma nova função que receba uma lista de palavras (não existe mais a noção de contexto) e imprima exemplos de sentenças em que todas as palavras estejam presentes. Lembre-se de destacar com apróstofes, nos exemplos retornados, as palavras passadas como parâmetro.
Exemplo de chamada:

exemplos_uso_palavras(['começar','dar'])

In [ ]:

def exemplos_uso_palavras(lista):
    print(f"palavras: {lista}")
    for sent in floresta.sents():
  
      if set(lista).issubset(sent):
 
            for i in range(len(sent)):
              if sent[i] in lista:
                sent[i] = "'" + sent[i] + "'"

            a = juntar(sent)
            print(a)
            #pos = sent.index(word)
            #left = " ".join(sent[:pos])
            #right = " ".join(sent[pos + 1:])
            #print(f"{left} '{word}' {right}")


exemplos_uso_palavras(['começar','dar'])

def juntar(lista):
    if not lista:
        return ''
    if len(lista) == 1:
        return lista[0]

    s = ' '.join(lista[:-1])
    return f'{s} {lista[-1]}'


palavras: ['começar', 'dar']
Mas é essa a ideia ; para_já promover este encontro , reunir ideias , depois se resultar material 'começar' a incorporar- lo e 'dar' forma a um ' site ' que tem sede simbólica em Sagres . »
